# Day 5: SHAP Analysis

## Learning Objectives
- Understand SHAP (SHapley Additive exPlanations)
- Implement SHAP-like analysis from scratch
- Interpret global and local explanations

---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("✅ Libraries loaded!")
print("📚 Day 5: SHAP Analysis")

✅ Libraries loaded!
📚 Day 5: SHAP Analysis


In [ ]:
# ============================================================
# PREPARE DATA
# ============================================================

print("PREPARING DATA")
print("="*60)

np.random.seed(42)
n_days = 1500

returns = np.random.normal(0.0002, 0.015, n_days)
for i in range(1, len(returns)):
    returns[i] += 0.05 * returns[i-1]

prices = 100 * np.cumprod(1 + returns)
df = pd.DataFrame({'price': prices, 'returns': returns})

# Create features
df['ret_1d'] = df['price'].pct_change(1)
df['ret_5d'] = df['price'].pct_change(5)
df['vol_5d'] = df['returns'].rolling(5).std()
df['vol_20d'] = df['returns'].rolling(20).std()
df['mom_5d'] = df['returns'].rolling(5).sum()
df['mom_20d'] = df['returns'].rolling(20).sum()

gains = df['returns'].where(df['returns'] > 0, 0).rolling(14).mean()
losses = (-df['returns'].where(df['returns'] < 0, 0)).rolling(14).mean()
df['rsi'] = 100 - (100 / (1 + gains / (losses + 1e-10)))

df['target'] = (df['returns'].shift(-1) > 0).astype(int)
df = df.dropna()

feature_cols = ['ret_1d', 'ret_5d', 'vol_5d', 'vol_20d', 'mom_5d', 'mom_20d', 'rsi']
X = df[feature_cols].values
y = df['target'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

split = int(len(X) * 0.7)
X_train, X_test = X_scaled[:split], X_scaled[split:]
y_train, y_test = y[:split], y[split:]

print(f"Features: {feature_cols}")

## Part 1: SHAP Concept

**SHAP values** use game theory to allocate prediction "credit" to features:

$$\phi_i = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(n-|S|-1)!}{n!} [f(S \cup \{i\}) - f(S)]$$

Key properties:
- **Local accuracy**: Sum of SHAP values = prediction - expected value
- **Missingness**: Features not in model get SHAP = 0
- **Consistency**: If a model relies more on feature, its SHAP increases

In [ ]:
# ============================================================
# TRAIN MODEL
# ============================================================

print("TRAINING MODEL")
print("="*60)

model = GradientBoostingClassifier(n_estimators=100, max_depth=4, random_state=42)
model.fit(X_train, y_train)

train_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)

print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")

## Part 2: SHAP-Like Implementation

In [ ]:
# ============================================================
# SIMPLIFIED SHAP CALCULATION
# ============================================================

print("CALCULATING SHAP-LIKE VALUES")
print("="*60)

def calculate_shap_values(model, X_background, X_explain, n_samples=100):
    """
    Calculate SHAP-like values using baseline comparison.
    
    For each feature, we measure how much the prediction changes
    when we replace that feature with background values.
    """
    n_explain = min(n_samples, len(X_explain))
    n_features = X_explain.shape[1]
    
    shap_values = np.zeros((n_explain, n_features))
    expected_value = model.predict_proba(X_background)[:, 1].mean()
    
    for i in range(n_explain):
        x = X_explain[i:i+1]
        original_pred = model.predict_proba(x)[0, 1]
        
        for j in range(n_features):
            # Replace feature j with background mean
            x_modified = x.copy()
            x_modified[0, j] = X_background[:, j].mean()
            
            modified_pred = model.predict_proba(x_modified)[0, 1]
            
            # SHAP value = contribution of this feature
            shap_values[i, j] = original_pred - modified_pred
    
    return shap_values, expected_value

# Calculate
shap_values, expected = calculate_shap_values(model, X_train, X_test, n_samples=200)

print(f"Expected value (base prediction): {expected:.3f}")
print(f"SHAP values shape: {shap_values.shape}")

## Part 3: Global Explanations

In [ ]:
# ============================================================
# GLOBAL FEATURE IMPORTANCE
# ============================================================

print("GLOBAL FEATURE IMPORTANCE (SHAP)")
print("="*60)

# Mean absolute SHAP
mean_abs_shap = np.abs(shap_values).mean(axis=0)
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'mean_shap': mean_abs_shap
}).sort_values('mean_shap', ascending=False)

print("\nFeature Importance (Mean |SHAP|):")
for _, row in importance_df.iterrows():
    print(f"  {row['feature']:<12} {row['mean_shap']:.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
importance_df.plot(x='feature', y='mean_shap', kind='barh', ax=axes[0],
                   color='steelblue', alpha=0.7, legend=False)
axes[0].set_xlabel('Mean |SHAP Value|')
axes[0].set_title('Global Feature Importance', fontweight='bold')
axes[0].invert_yaxis()

# Summary plot (beeswarm)
for i, feat in enumerate(importance_df['feature'].values):
    feat_idx = feature_cols.index(feat)
    shap_vals = shap_values[:, feat_idx]
    feat_vals = X_test[:len(shap_vals), feat_idx]
    
    # Normalize for color
    norm_feat = (feat_vals - feat_vals.min()) / (feat_vals.max() - feat_vals.min() + 1e-10)
    
    y_jitter = i + np.random.randn(len(shap_vals)) * 0.15
    axes[1].scatter(shap_vals, y_jitter, c=norm_feat, cmap='coolwarm', alpha=0.5, s=15)

axes[1].set_yticks(range(len(feature_cols)))
axes[1].set_yticklabels(importance_df['feature'].values)
axes[1].axvline(0, color='gray', linewidth=0.5)
axes[1].set_xlabel('SHAP Value')
axes[1].set_title('SHAP Summary (Blue=Low, Red=High)', fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## Part 4: Local Explanations

In [ ]:
# ============================================================
# LOCAL EXPLANATION (SINGLE PREDICTION)
# ============================================================

print("LOCAL EXPLANATION (SINGLE PREDICTION)")
print("="*60)

def explain_prediction(idx):
    """Explain a single prediction."""
    sample = X_test[idx:idx+1]
    pred_prob = model.predict_proba(sample)[0, 1]
    actual = y_test[idx]
    sample_shap = shap_values[idx]
    
    print(f"\nSample {idx}:")
    print(f"  Prediction: {pred_prob:.2%} probability of UP")
    print(f"  Actual: {'UP' if actual == 1 else 'DOWN'}")
    print(f"  Base value: {expected:.2%}")
    print(f"\n  Feature contributions:")
    
    contrib_df = pd.DataFrame({
        'feature': feature_cols,
        'value': sample[0],
        'shap': sample_shap
    }).sort_values('shap', key=abs, ascending=False)
    
    for _, row in contrib_df.iterrows():
        direction = '+' if row['shap'] > 0 else '-'
        print(f"    {row['feature']:<12} = {row['value']:>7.3f}  {direction}{abs(row['shap']):.4f}")
    
    return contrib_df

# Explain a few predictions
contrib = explain_prediction(50)

In [ ]:
# ============================================================
# WATERFALL PLOT
# ============================================================

def waterfall_plot(idx, ax=None):
    """Create a waterfall plot for prediction explanation."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    
    sample_shap = shap_values[idx]
    pred_prob = model.predict_proba(X_test[idx:idx+1])[0, 1]
    
    # Sort by absolute contribution
    sorted_idx = np.argsort(np.abs(sample_shap))[::-1]
    
    cumulative = expected
    y_positions = []
    
    # Base
    ax.barh(0, expected, color='gray', alpha=0.5, height=0.6)
    ax.text(expected/2, 0, f'Base: {expected:.3f}', ha='center', va='center')
    
    for i, feat_idx in enumerate(sorted_idx[:6]):  # Top 6 features
        shap_val = sample_shap[feat_idx]
        color = 'green' if shap_val > 0 else 'red'
        
        ax.barh(i+1, shap_val, left=cumulative, color=color, alpha=0.6, height=0.6)
        ax.text(cumulative + shap_val/2, i+1, f'{shap_val:+.3f}', ha='center', va='center', fontsize=9)
        
        cumulative += shap_val
    
    # Final prediction
    ax.barh(len(sorted_idx[:6])+1, cumulative, color='steelblue', alpha=0.7, height=0.6)
    ax.text(cumulative/2, len(sorted_idx[:6])+1, f'Pred: {pred_prob:.3f}', ha='center', va='center')
    
    labels = ['Base'] + [feature_cols[i] for i in sorted_idx[:6]] + ['Prediction']
    ax.set_yticks(range(len(labels)))
    ax.set_yticklabels(labels)
    ax.set_xlabel('Probability')
    ax.set_title(f'Prediction Explanation (Sample {idx})', fontweight='bold')
    ax.axvline(0.5, color='gray', linestyle='--', alpha=0.5)
    
    return ax

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
waterfall_plot(50, axes[0])
waterfall_plot(100, axes[1])
plt.tight_layout()
plt.show()

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║               DAY 5 COMPLETE: SHAP ANALYSIS                      ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ SHAP concept and game-theoretic foundations                  ║
║  ✓ Simplified SHAP implementation                               ║
║  ✓ Global explanations (feature importance)                     ║
║  ✓ Local explanations (individual predictions)                  ║
║  ✓ Summary and waterfall plots                                  ║
╚══════════════════════════════════════════════════════════════════╝

Tomorrow: Day 6 - Permutation & Partial Dependence
""")